In [1]:
%matplotlib inline

In [2]:
from __future__ import division,print_function

import os, json, shutil
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
import pandas as pd
import sys
import random

# created a file called 'utils.py' to store convenience functions.

In [3]:
import utils; reload(utils)
from utils import *

ImportError: No module named utils

In [3]:
# point to your training images
train_dir = '/home/ubuntu/nbs/data/statefarm/train'
# point to the 'driver_imgs_list.csv'
lookup = '/home/ubuntu/nbs/data/statefarm/driver_imgs_list.csv'
# point to the validation directory, which will be created in the next block
val_dir = '/home/ubuntu/nbs/data/statefarm/valid'

In [4]:
os.getcwd()

'/home/ubuntu/nbs'

In [17]:
# creates a directory, called 'validation', and into it, creates 10 subdirs, one for each class.
directory = 'valid'
if not os.path.exists(directory):
    os.makedirs(directory)
cwd = os.getcwd()
path = val_dir
[os.mkdir('{}/c{}'.format(path, i)) for i in range(10)]

[None, None, None, None, None, None, None, None, None, None]

In [10]:
# read in the 'driver_imgs_list.csv' file
df = pd.read_csv(lookup)

In [6]:
df.head(3)

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg


In [11]:
# There are 26 unique drivers in the train set. 
# We want 20%, so pick 5 at random
driver_names = ['p002', 'p012', 'p014', 'p015', 'p016', 'p021', 'p022', 'p024', 'p026', 'p035', 'p039', 'p041', 'p042', 'p045', 'p047', 'p049', 'p050', 'p051', 'p052', 'p056', 'p061', 'p064', 'p066', 'p072', 'p075', 'p081']
driver_names_for_validation = random.sample(driver_names,5)

In [8]:
# filter df by selecting rows with the subjects we want
df = df[df['subject'].isin(driver_names_for_validation)]

NameError: name 'driver_names_for_validation' is not defined

In [34]:
# do the move here:
i = 0
for index, row in df.iterrows():
    to_move = train_dir + '/' +  row['classname'] + '/' + row['img']
    #print to_move
    i = i + 1
    move_to = val_dir + '/' +  row['classname']
    #print move_to
    shutil.move(to_move, move_to)

In [17]:
df.shape[:]

(22424, 3)

In [35]:
print('files to move: ' ,df.shape[0])
print('files moved:   ' ,i)

files to move:  4285
files moved:    4285


In [18]:
current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir+'/data/statefarm'

In [19]:
%cd $DATA_HOME_DIR
#Set path to sample/ path if desired
path = DATA_HOME_DIR + '/' #'/sample/'
test_path = DATA_HOME_DIR + '/test/' #We use all the test data
results_path=DATA_HOME_DIR + '/results/'
train_path=path + '/train/'
valid_path=path + '/valid/'
model_path = path + 'models/'

/home/ubuntu/nbs/data/statefarm


In [20]:
  path = "data/statefarm/"
  results_path = "data/statefarm/results/"  
# path = "data/dogscatsredux/sample/"

# Use a pretrained VGG model with **Vgg16** class

Our first step is simply to use a model that has been fully created for us, which can recognise a wide variety (1,000 categories) of images. We will use 'VGG', which won the 2014 Imagenet competition, and is a very simple model to create and understand. The VGG Imagenet team created both a larger, slower, slightly more accurate model (*VGG  19*) and a smaller, faster model (*VGG 16*). We will be using VGG 16 since the much slower performance of VGG19 is generally not worth the very minor improvement in accuracy.

We have created a python class, *Vgg16*, which makes using the VGG 16 model very straightforward. 

In [4]:
# As large as you can, but no larger than 64 is recommended. 
# If you have an older or cheaper GPU, you'll run out of memory, so will have to decrease this.
batch_size=64

In [21]:
# Import our class, and instantiate
#import vgg16; reload(vgg16)
#from vgg16 import Vgg16
from vgg16bn import Vgg16BN

In [22]:
vgg = Vgg16BN()
model=vgg.model

In [7]:
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

In [8]:
conv_model = Sequential(conv_layers)

In [9]:
# batches shuffle must be set to False when pre-computing features
batches = vgg.get_batches(path+'train', batch_size=batch_size, shuffle=False)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)
test_batches = vgg.get_batches(path+'test', batch_size=batch_size, shuffle=False)

Found 18139 images belonging to 10 classes.
Found 4285 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [10]:
(val_classes, trn_classes, val_labels, trn_labels, 
    val_filenames, filenames, test_filenames) = get_classes(path)

Found 18139 images belonging to 10 classes.
Found 4285 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [12]:
conv_feat = conv_model.predict_generator(batches, batches.nb_sample)
save_array(path+'results/conv_feat.dat', conv_feat)

In [13]:
conv_val_feat = conv_model.predict_generator(val_batches, val_batches.nb_sample)
save_array(path+'results/conv_val_feat.dat', conv_val_feat)

In [26]:
conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)
save_array(path+'results/conv_test_feat.dat', conv_test_feat)

In [11]:
conv_feat = load_array(path+'results/conv_feat.dat')
conv_val_feat = load_array(path+'results/conv_val_feat.dat')

In [12]:
conv_test_feat = load_array(path+'results/conv_test_feat.dat')

In [32]:
def get_bn_layers(p):
    model = Sequential( [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p/2),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        Dropout(p/2),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ])
    model.compile(Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [33]:
p=0.95

In [34]:
model = get_bn_layers(p)

In [35]:
model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=5, 
             validation_data=(conv_val_feat, val_labels))

Train on 18139 samples, validate on 4285 samples
Epoch 1/5
18139/18139 [==============================] - 15s - loss: 3.8609 - acc: 0.4367 - val_loss: 1.0032 - val_acc: 0.7260
Epoch 2/5
18139/18139 [==============================] - 15s - loss: 0.5034 - acc: 0.8678 - val_loss: 1.1222 - val_acc: 0.7587
Epoch 3/5
18139/18139 [==============================] - 15s - loss: 0.2158 - acc: 0.9385 - val_loss: 0.8781 - val_acc: 0.7827
Epoch 4/5
18139/18139 [==============================] - 15s - loss: 0.1244 - acc: 0.9640 - val_loss: 0.8472 - val_acc: 0.7998
Epoch 5/5
18139/18139 [==============================] - 15s - loss: 0.0986 - acc: 0.9708 - val_loss: 0.9192 - val_acc: 0.7708


In [36]:
model.optimizer.lr=0.1

In [37]:
model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=2, 
             validation_data=(conv_val_feat, val_labels))

Train on 18139 samples, validate on 4285 samples
Epoch 1/2
18139/18139 [==============================] - 15s - loss: 0.0834 - acc: 0.9738 - val_loss: 0.8113 - val_acc: 0.8033
Epoch 2/2
18139/18139 [==============================] - 15s - loss: 0.0710 - acc: 0.9772 - val_loss: 0.9061 - val_acc: 0.7916


In [38]:
model.optimizer.lr=0.01

In [39]:
model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=3, 
             validation_data=(conv_val_feat, val_labels))

Train on 18139 samples, validate on 4285 samples
Epoch 1/3
18139/18139 [==============================] - 15s - loss: 0.0651 - acc: 0.9814 - val_loss: 1.0106 - val_acc: 0.7564
Epoch 2/3
18139/18139 [==============================] - 15s - loss: 0.0597 - acc: 0.9816 - val_loss: 1.0413 - val_acc: 0.7732
Epoch 3/3
18139/18139 [==============================] - 15s - loss: 0.0557 - acc: 0.9834 - val_loss: 0.8192 - val_acc: 0.8021


In [41]:
model.optimizer.lr=0.001

In [42]:
model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=3, 
             validation_data=(conv_val_feat, val_labels))

Train on 18139 samples, validate on 4285 samples
Epoch 1/3
18139/18139 [==============================] - 15s - loss: 0.0581 - acc: 0.9824 - val_loss: 1.0302 - val_acc: 0.7769
Epoch 2/3
18139/18139 [==============================] - 15s - loss: 0.0523 - acc: 0.9847 - val_loss: 1.0706 - val_acc: 0.7673
Epoch 3/3
18139/18139 [==============================] - 15s - loss: 0.0435 - acc: 0.9875 - val_loss: 1.1860 - val_acc: 0.7692


In [43]:
def fit_model():
    model = get_bn_layers(p)
    model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=4, 
             validation_data=(conv_val_feat, val_labels))
    model.optimizer.lr=0.1
    model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=1, 
             validation_data=(conv_val_feat, val_labels))
    model.optimizer.lr=0.01
    model.fit(conv_feat, trn_labels, batch_size=batch_size, nb_epoch=3, 
             validation_data=(conv_val_feat, val_labels))
    return model

In [45]:
models = [fit_model() for i in range(6)]

Train on 18139 samples, validate on 4285 samples
Epoch 1/4
18139/18139 [==============================] - 15s - loss: 3.8196 - acc: 0.4442 - val_loss: 0.8214 - val_acc: 0.7683
Epoch 2/4
18139/18139 [==============================] - 15s - loss: 0.5007 - acc: 0.8696 - val_loss: 0.7264 - val_acc: 0.7965
Epoch 3/4
18139/18139 [==============================] - 15s - loss: 0.2139 - acc: 0.9411 - val_loss: 0.8832 - val_acc: 0.7652
Epoch 4/4
18139/18139 [==============================] - 15s - loss: 0.1291 - acc: 0.9619 - val_loss: 0.9563 - val_acc: 0.7634
Train on 18139 samples, validate on 4285 samples
Epoch 1/1
18139/18139 [==============================] - 15s - loss: 0.0970 - acc: 0.9716 - val_loss: 0.9099 - val_acc: 0.7809
Train on 18139 samples, validate on 4285 samples
Epoch 1/3
18139/18139 [==============================] - 15s - loss: 0.0696 - acc: 0.9782 - val_loss: 0.7867 - val_acc: 0.8070
Epoch 2/3
18139/18139 [==============================] - 15s - loss: 0.0643 - acc: 0.9810 -

In [47]:
model_path = path + 'models/'

In [48]:
for i,m in enumerate(models):
    m.save_weights(model_path+'fish-'+str(i)+'.pkl')

In [ ]:
for i,m in enumerate(models):
    m.load_weights(model_path+'fish-'+str(i)+'.pkl')

In [49]:
evals = np.array([m.evaluate(conv_val_feat, val_labels, batch_size=batch_size) for m in models])

4285/4285 [==============================] - 1s     


In [50]:
evals

array([[ 0.8806,  0.7853],
       [ 1.1523,  0.7531],
       [ 1.1068,  0.7631],
       [ 1.0755,  0.7855],
       [ 1.0137,  0.768 ],
       [ 0.9281,  0.7764]])

In [51]:
evals.mean(axis=0)

array([ 1.0262,  0.7719])

In [52]:
all_preds = np.stack([m.predict(conv_test_feat, batch_size=batch_size) for m in models])
save_array(model_path+'allpreds.dat', all_preds)

In [29]:
load_array(model_path+'allpreds.dat', all_preds)

NameError: name 'all_preds' is not defined

In [54]:
avg_preds = all_preds.mean(axis=0)

In [70]:
avg_preds.shape

(79726, 10)

In [72]:
all_preds.shape

(6, 79726, 10)

In [71]:
val_classes.shape

(4285,)

In [56]:
filenames = test_batches.filenames

In [ ]:
keras.metrics.categorical_accuracy(, avg_preds).eval()

In [19]:
len(filenames)

79726

In [20]:
preds.shape

(79726, 10)

In [58]:
subm = do_clip(avg_preds,0.7719)

In [59]:
subm[:5]

array([[ 0.0264,  0.0264,  0.0264,  0.0264,  0.0264,  0.0264,  0.0264,  0.2709,  0.0264,  0.5177],
       [ 0.39  ,  0.0259,  0.0259,  0.0259,  0.0259,  0.0259,  0.0259,  0.0259,  0.0259,  0.403 ],
       [ 0.0299,  0.0299,  0.0299,  0.0299,  0.7084,  0.0299,  0.0299,  0.0299,  0.0523,  0.0299],
       [ 0.0287,  0.0287,  0.0287,  0.0287,  0.0287,  0.0287,  0.0903,  0.0287,  0.68  ,  0.0287],
       [ 0.028 ,  0.0902,  0.028 ,  0.028 ,  0.028 ,  0.028 ,  0.028 ,  0.0347,  0.3544,  0.3529]], dtype=float32)

In [60]:
ids = np.array([f[8:] for f in filenames])

In [61]:
ids[:5]

array(['img_81601.jpg', 'img_14887.jpg', 'img_62885.jpg', 'img_45125.jpg', 'img_22633.jpg'], 
      dtype='|S14')

In [62]:
submm=np.stack([ids,subm[:,0],subm[:,1],subm[:,2],subm[:,3],subm[:,4],subm[:,5],subm[:,6],subm[:,7],subm[:,8],subm[:,9]],axis=1)

In [63]:
submm[:3]

array([['img_81601.jpg', '0.0264233928174', '0.0264233928174', '0.0264233928174', '0.0264233928174',
        '0.0264233928174', '0.0264233928174', '0.0264233928174', '0.270909547806',
        '0.0264233928174', '0.517703354359'],
       ['img_14887.jpg', '0.389978826046', '0.0258737504482', '0.0258737504482', '0.0258737504482',
        '0.0258737504482', '0.0258737504482', '0.0258737504482', '0.0258737504482',
        '0.0258737504482', '0.403031110764'],
       ['img_62885.jpg', '0.0299060307443', '0.0299060307443', '0.0299060307443', '0.0299060307443',
        '0.708422899246', '0.0299060307443', '0.0299060307443', '0.0299060307443',
        '0.0523288920522', '0.0299060307443']], 
      dtype='|S32')

In [64]:
import pandas as pd 
df = pd.DataFrame(submm)

In [65]:
df.insert(0, 'img', [a[4:] for a in test_filenames])

In [66]:
df.to_csv(path+"submission_sf_ens.csv",index=False)

In [67]:
len(df)

79726

In [68]:
df.head()

,img,0,1,2,3,4,5,6,7,8,9,10
0,own/img_81601.jpg,img_81601.jpg,0.0264233928174,0.0264233928174,0.0264233928174,0.0264233928174,0.0264233928174,0.0264233928174,0.0264233928174,0.270909547806,0.0264233928174,0.517703354359
1,own/img_14887.jpg,img_14887.jpg,0.389978826046,0.0258737504482,0.0258737504482,0.0258737504482,0.0258737504482,0.0258737504482,0.0258737504482,0.0258737504482,0.0258737504482,0.403031110764
2,own/img_62885.jpg,img_62885.jpg,0.0299060307443,0.0299060307443,0.0299060307443,0.0299060307443,0.708422899246,0.0299060307443,0.0299060307443,0.0299060307443,0.0523288920522,0.0299060307443
3,own/img_45125.jpg,img_45125.jpg,0.028706651181,0.028706651181,0.028706651181,0.028706651181,0.028706651181,0.028706651181,0.0903350710869,0.028706651181,0.680011630058,0.028706651181
4,own/img_22633.jpg,img_22633.jpg,0.0279557984322,0.0902168154716,0.0279557984322,0.0279557984322,0.0279557984322,0.0279557984322,0.0279557984322,0.0347320996225,0.354382961988,0.352933317423


In [69]:
from IPython.display import FileLink
FileLink(path+'submission_sf_ens.csv')

/home/ubuntu/nbs/data/statefarm/submission_sf_ens.csv